In [1]:
import pandas as pd
debt_data=pd.read_csv("fedex_debt_data.csv")
debt_data

,Customer_ID,Amount_Due,Days_Overdue,Credit_Score,Past_Defaults,Calls_Made,Assigned_Agency,Recovery_Probability,Target_Paid
0,CUST_0000,122958,226,819,1,19,Beta Recovery,0.60,0
1,CUST_0001,147867,351,326,0,14,Gamma Legal,0.01,0
2,CUST_0002,132932,324,498,1,15,Alpha Collections,0.01,0
3,CUST_0003,366838,292,686,0,12,Gamma Legal,0.30,1
4,CUST_0004,260178,186,648,1,16,Gamma Legal,0.30,0
...,...,...,...,...,...,...,...,...,...
995,CUST_0995,113816,81,725,0,1,NaN,0.50,0
996,CUST_0996,129778,69,574,0,1,Gamma Legal,0.50,1
997,CUST_0997,63292,154,583,0,16,NaN,0.50,1
998,CUST_0998,286977,102,472,0,15,Alpha Collections,0.20,1


In [2]:
debt_data.isnull().sum()

Customer_ID               0
Amount_Due                0
Days_Overdue              0
Credit_Score              0
Past_Defaults             0
Calls_Made                0
Assigned_Agency         247
Recovery_Probability      0
Target_Paid               0
dtype: int64

In [3]:
debt_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Customer_ID           1000 non-null   object 
 1   Amount_Due            1000 non-null   int64  
 2   Days_Overdue          1000 non-null   int64  
 3   Credit_Score          1000 non-null   int64  
 4   Past_Defaults         1000 non-null   int64  
 5   Calls_Made            1000 non-null   int64  
 6   Assigned_Agency       753 non-null    object 
 7   Recovery_Probability  1000 non-null   float64
 8   Target_Paid           1000 non-null   int64  
dtypes: float64(1), int64(6), object(2)
memory usage: 70.4+ KB


In [4]:
debt_data["Assigned_Agency"].value_counts()

Assigned_Agency
Gamma Legal          261
Beta Recovery        248
Alpha Collections    244
Name: count, dtype: int64

In [5]:
X=debt_data.drop(["Customer_ID","Assigned_Agency","Target_Paid","Recovery_Probability"],axis=1)
y=debt_data["Target_Paid"]

In [6]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
model_pipeline=Pipeline(
    steps=[
        ('imputer',IterativeImputer(max_iter=10,random_state=42)),
        ('scaler',StandardScaler()),
        ('model', RandomForestClassifier(n_estimators=100, random_state=42))
          ])


In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
model_pipeline.fit(X_train,y_train)
from sklearn.metrics import accuracy_score, classification_report
preds = model_pipeline.predict(X_test)
print("Accuracy:", accuracy_score(y_test, preds))
print("\nClassification Report:\n", classification_report(y_test, preds))

Accuracy: 0.69

Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.81      0.78       138
           1       0.50      0.42      0.46        62

    accuracy                           0.69       200
   macro avg       0.63      0.62      0.62       200
weighted avg       0.68      0.69      0.68       200



In [8]:
import joblib
joblib.dump(model_pipeline,"debt_pred_pipeline.pkl")
print("Saved the pipeline")

Saved the pipeline


In [9]:
from xgboost import XGBClassifier
model_2=XGBClassifier(
        n_estimators=100, 
        learning_rate=0.1, 
        max_depth=5, 
        random_state=42,
        use_label_encoder=False,
        eval_metric='logloss'
    )
model_pipeline=Pipeline(
    steps=[
        ('imputer',IterativeImputer(max_iter=10,random_state=42)),
        ('scaler',StandardScaler()),
        ('model', model_2)
          ])


In [10]:
model_pipeline.fit(X_train,y_train)
from sklearn.metrics import accuracy_score, classification_report
preds = model_pipeline.predict(X_test)
print("Accuracy:", accuracy_score(y_test, preds))
print("\nClassification Report:\n", classification_report(y_test, preds))

Accuracy: 0.73

Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.82      0.81       138
           1       0.57      0.53      0.55        62

    accuracy                           0.73       200
   macro avg       0.68      0.68      0.68       200
weighted avg       0.73      0.73      0.73       200



/Users/vishwa/miniforge3/envs/myenv/lib/python3.11/site-packages/xgboost/training.py:199: UserWarning: [23:11:41] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1765326969566/work/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


73% accuracy is much better than the previous 69%

Looking at precision, It is 0.80 for class 0, which means when our model says they won't pay, It is right 80% of time

In [11]:
import joblib
joblib.dump(model_pipeline,"debt_pred_pipeline.pkl")
print("Saved the pipeline")

Saved the pipeline
